# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = 'output_data/cities.csv'
weather_data_df = pd.read_csv(csv_file)
weather_data_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,vaini,75,TO,1584154254,83,-21.20,-175.20,80.600,6.70
1,qaqortoq,100,GL,1584154257,79,60.72,-46.03,21.200,1.50
2,kropotkin,35,RU,1584154330,69,45.44,40.58,52.160,3.70
3,sao jose da coroa grande,100,BR,1584154331,72,-8.90,-35.15,81.914,4.39
4,hithadhoo,20,MV,1584154253,58,-0.60,73.08,83.642,3.26


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Grab values for coordinates and humidities
coordinates = weather_data_df[["Lat", "Lng"]].astype(float)
humidities = weather_data_df["Humidity"].astype(int)

In [4]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidities, dissipating=False, 
                                    max_intensity=10, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Clean up data to meet the ideal conditions for temp, cloudiness and wind speed.
ideal_weather_df = weather_data_df.loc[(weather_data_df['Max Temp'] > 70) & (weather_data_df['Max Temp'] < 80)]
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Wind Speed'] < 10) & (ideal_weather_df['Cloudiness'] == 0)]
ideal_weather_df = ideal_weather_df.dropna(how='any')

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store new dataframe into a hotel_df, reset the index and add a new column to hold the Hotel Names.
hotel_df = ideal_weather_df.reset_index()
hotel_df["Hotel Name"] =  ""
hotel_df.head()

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,40,arraial do cabo,0,BR,1584154362,86,-22.97,-42.03,77.324,5.19,
1,48,mar del plata,0,AR,1584154190,78,-38.00,-57.56,73.994,5.10,
2,159,gorom-gorom,0,BF,1584154419,11,14.44,-0.24,74.822,4.52,
3,161,parbhani,0,IN,1584154458,19,19.27,76.78,76.910,4.79,
4,180,chuy,0,UY,1584154264,82,-33.70,-53.46,70.808,5.25,


In [7]:
# Grab the lat longs for each city
lats = hotel_df["Lat"]
lngs = hotel_df["Lng"]

In [8]:
# Create a request URL for each lat/long to pull 1st hotel (logding) within a 5000 meter radius
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?&type=lodging&key='
radius = '&radius=5000'
location = '&location='
request_url = base_url + g_key + radius + location

In [9]:
# Iterrate through the hotel_df and update the hotel name column
for index, row in hotel_df.iterrows():
    response = requests.get(request_url+str(lats[index])+','+str(lngs[index])).json()
    hotel_name = response["results"][0]["name"]
    hotel_df.at[index,'Hotel Name'] = hotel_name

IndexError: list index out of range

In [10]:
hotel_df.head()

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,40,arraial do cabo,0,BR,1584154362,86,-22.97,-42.03,77.324,5.19,
1,48,mar del plata,0,AR,1584154190,78,-38.00,-57.56,73.994,5.10,
2,159,gorom-gorom,0,BF,1584154419,11,14.44,-0.24,74.822,4.52,
3,161,parbhani,0,IN,1584154458,19,19.27,76.78,76.910,4.79,
4,180,chuy,0,UY,1584154264,82,-33.70,-53.46,70.808,5.25,


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig
# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…